In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

import tams
from tams.idealized import Blob, Field, Sim

In [ ]:
Blob().get_tendency()

In [ ]:
Blob().polygon

In [ ]:
Blob(c=(1, 2)).split()

In [ ]:
Field(Blob(c=(1, 2), a=3, b=1, theta=20).split(3)).to_geopandas().plot(fc="none")

In [ ]:
b1 = Blob(a=3, b=1, theta=30)
b2 = Blob(a=4, b=2)
b3 = b1.merge(b2)

Field([b1, b2, b3]).to_geopandas().plot(fc="none")

In [ ]:
Field()

In [ ]:
%%time

b = Blob(c=(0, 0), a=5, b=3)

ctt = Field(b).to_xarray()

(ce,), _ = tams.identify(ctt)

fig, ax = plt.subplots(figsize=(10, 6))

ctt.plot(ax=ax)
gpd.GeoSeries(b.ring).plot(ec="magenta", ax=ax)

ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none", zorder=10)

ax.set_aspect("equal")

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True, layout="constrained")

ctt.sel(lat=0, method="nearest").plot(ax=ax1)
ctt.sel(lon=0, method="nearest").plot(ax=ax2)

ax2.set_ylabel("")
for ax in fig.get_axes():
    ax.axhline(235, c="0.35", ls=":")

In [ ]:
# Flipping depth and making CTT threshold greater than the background,
# we can get a reverse well.
ctt = Field(
    Blob(c=(0, 0), a=5, b=3, depth=-20)
).to_xarray(ctt_threshold=300)

# TAMS will still identify if we adjust the thresholds accordingly
(ce,), _ = tams.identify(ctt, ctt_threshold=300, ctt_core_threshold=310)

fig, ax = plt.subplots()

ctt.plot(ax=ax)
ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none")

In [ ]:
fld = Field(
    [
        Blob(c=(0, 0), a=5, b=3),
        Blob(c=(5, 0), a=5, b=3),
    ]
)

fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(8, 6), sharex=True, sharey=True, layout="constrained")

kws = dict(vmin=170, vmax=270)
for ax, ctt in zip([ax1, ax2], [fld.to_xarray(), fld.to_xarray(additive=True)]):
    ctt.plot(ax=ax, **kws)
    ax.text(0.01, 0.98, f"min: {ctt.min().item():.2f}", ha="left", va="top", transform=ax.transAxes)
    (ce,), _ = tams.identify(ctt)
    ce.plot(ax=ax, lw=2.5, ls=":", ec="black", fc="none")

ax1.set_xlabel("")

for ax in [ax1, ax2]:
    ax.set_aspect("equal")

In [ ]:
# Nothing
Sim().advance(2).to_xarray().plot(col="time")

In [ ]:
%%time

ctt = Sim(
    Field(
        [
            Blob(a=5).set_tendency(c=(2, 0)),
            Blob(c=(-20, -10), a=3).set_tendency(c=(5, 3), a=1),
        ],
        lat=(-15, 15, 61),
    ),
).advance(9).to_xarray()

ces, _ = tams.identify(ctt)

fg = ctt.plot(col="time", cmap="inferno_r", col_wrap=5, aspect=1.35)
for ce, ax in zip(ces, fg.axs.flat):
    ce.plot(ax=ax, lw=2, ec="w", fc="none")